In [ ]:
# installations
%pip install langchain==0.1.0 | tail -n 1
%pip install langchain-ibm==0.0.8 | tail -n 1
%pip install langchain-community==0.1.0 | tail -n 1
%pip install langchain-core==0.1.0 | tail -n 1
%pip install ibm-watsonx-ai | tail -n 1
%pip install chromadb | tail -n 1
%pip install tiktoken | tail -n 1
%pip install bs4 | tail -n 1
%pip install -U langchain-ollama

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-ollama 0.3.2 requires langchain-core<1.0.0,>=0.3.52, but you have langchain-core 0.1.23 which is incompatible.
langchain-mistralai 0.2.9 requires langchain-core<1.0.0,>=0.3.47, but you have langchain-core 0.1.23 which is incompatible.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.1.23 which is incompatible.
langchain-ibm 0.3.10 requires langchain-core<0.4.0,>=0.3.39, but you have langchain-core 0.1.23 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement langchain-ibm==0.0.8 (from versions: 0.0.1, 0.1.0, 0.1.1, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.1.8, 0.1.9rc0, 0.1.9, 0.1.10, 0.1.11, 0.1.

In [ ]:
# imports
import getpass

from langchain.vectorstores import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough

/opt/anaconda3/lib/python3.12/site-packages/langchain_community/document_loaders/blob_loaders/file_system.py:6: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.document_loaders.blob_loaders.schema import Blob, BlobLoader
/opt/anaconda3/lib/python3.12/site-packages/langchain_community/document_loaders/__init__.py:227: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shi

RuntimeError: no validator found for <class 'langchain_core.prompts.prompt.PromptTemplate'>, see `arbitrary_types_allowed` in Config

In [7]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model = "stablelm2",
    temperature = 0.8,
    # num_predict = 256,
    # other params ...
)

In [8]:
from langchain_core.prompts import PromptTemplate

template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

In [8]:
agent = prompt | llm

In [9]:

agent.invoke({"query": "What sport is played at the US Open?"})

AIMessage(content='The sport that is predominantly played at the U.S. Open, a major professional tennis tournament, is tennis. The U.S. Open, first held in 1877 and now called the US Open Series, is one of the four Grand Slam tournaments in tennis.\n\nTennis, as we know it today, was not invented until the late 19th century by professionals and amateur players alike. However, the sport did not become popular enough to be considered a major international sporting event until the mid-20th century. As a result, competitions were initially limited to club and amateur levels.\n\nThe U.S Open, as well as other important tournaments such as Wimbledon and the Australian Open, serves as a platform for professional tennis players to showcase their skills on an international level. The tournament has grown in popularity over the years due to its prestigious status among tennis enthusiasts worldwide.\n\nIn addition to tennis, there are also events within the U.S. Open Series that involve other spo

In [10]:
agent.invoke({"query": "Where was the 2024 US Open Tennis Championship?"})

AIMessage(content='The U.S. Open Tennis Championships, also known as the Grand Slam, is one of the four major tennis championships held annually on the United States mainland. The tournament takes place at different venues throughout the years due to its location changeover.\n\nIn 2021, the U.S. Open took place in New York City and Queens from September 14th to October 4th, at the USTA Billie Jean King Center (formerly Madison Square Garden) and the Grand Hyatt New York, respectively.\n\nHowever, it\'s essential to note that since 2016, due to financial and logistical reasons, the tournament has been held in Flushing Meadows-Cornerstone Park, near the site of the former U.S. Open venue at the World Court. In this case, the event was named the "US Open" for short periods during its transition.\n\nSo, to summarize: The 2021 U.S. Open Tennis Championship took place in New York City and Queens (USTA Billie Jean King Center and Grand Hyatt New York). For events that occurred before 2021 (li

In [9]:
urls = [
    "https://www.ibm.com/case-studies/us-open",
    "https://www.ibm.com/sports/usopen",
    "https://newsroom.ibm.com/US-Open-AI-Tennis-Fan-Engagement",
    "https://newsroom.ibm.com/2024-08-15-ibm-and-the-usta-serve-up-new-and-enhanced-generative-ai-features-for-2024-us-open-digital-platforms",
]

In [10]:
from langchain_community.document_loaders import WebBaseLoader

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
docs_list[0]

Document(metadata={'source': 'https://www.ibm.com/case-studies/us-open', 'title': 'U.S. Open | IBM', 'description': 'To help the US Open stay on the cutting edge of customer experience, IBM Consulting built powerful generative AI models with watsonx.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nU.S. Open | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\n\nCase Studies\n\n\n\n\nUS Open \n\n\n\n\n\n\n                \n\n\n\n  \n    Acing the US Open digital experience\n\n\n\n\n\n\n    \n\n\n            \n\n                    \n\n\n  \n  \n      AI models built with watsonx transform data into insight\n  \n\n\n\n\n    \n\n\n                \n\n\nGet the latest AI and tech insights\n\n\n\nLearn More\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFor two weeks at the end of summer, nearly one million people 

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [ ]:
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
)

In [13]:
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [ ]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="agentic-rag-chroma",

)

TypeError: Chroma.__init__() got an unexpected keyword argument 'embedding_functions'

In [ ]:

retriever = vectorstore.as_retriever()